In [1]:
import numpy as np
from IPython.display import display, Image
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import pandas as pd
from hmmlearn import hmm
import numpy as np
import seaborn as sns
from scipy.stats import norm
import itertools
import matplotlib.pyplot as plt
from hmmlearn.hmm import GMMHMM
from  sklearn.metrics import mean_squared_error as mse
from astropy.io import fits
import matplotlib.pyplot as plt
import os, sys
dirpath = os.getcwd().split("code")[0]+"code/"
sys.path.append(dirpath)
from pre_process import clean_LC

def plot_state_matrix(cm, states,title='Transition Probabilities',cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(states))
    plt.xticks(tick_marks, states)
    plt.yticks(tick_marks, states)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.3f'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > cm.max() / 2. else "black")
    plt.ylabel('Origin state')
    plt.xlabel('Destination state')
    plt.tight_layout()

    
def plot_model_and_data(plot_lc, model, p_n=2000, real_lc=[] ):
    X_state = model.predict(plot_lc) #predice el estado, no el valor 
    X_state_mean = model.means_[X_state][:,0]
    X_state_cov = np.sqrt(model.covars_[X_state][:,0,0])
    if len(real_lc) != 0:
        if len(plot_lc) != len(real_lc):
            mask_nan = np.isnan(real_lc)
            aux = np.tile(np.nan, len(real_lc) )
            
            aux_lc = aux.copy()
            aux_lc[~mask_nan] = np.squeeze(plot_lc)
            plot_lc = aux_lc
            
            aux_X_state_mean = aux.copy()
            aux_X_state_mean[~mask_nan] = X_state_mean
            X_state_mean = aux_X_state_mean
            
            aux_X_state_cov = aux.copy()
            aux_X_state_cov[~mask_nan] = X_state_cov
            X_state_cov = aux_X_state_cov
    plt.figure(figsize=(14,6))
    plt.plot(plot_lc,'r.-',label="Data")
    plt.fill_between(np.arange(len(plot_lc)),
                     X_state_mean-1.96*X_state_cov,
                     X_state_mean+1.96*X_state_cov) 
    plt.plot(X_state_mean,'g*-',label="Prediccion de la media") 
    plt.legend()
    plt.title("prediccion de markov en curva")
    plt.show()
    
    plt.figure(figsize=(14,6))
    plt.plot(plot_lc[:p_n],'r.-',label="Data")
    plt.fill_between(np.arange(p_n),
                     X_state_mean[:p_n]-1.96*X_state_cov[:p_n],
                     X_state_mean[:p_n]+1.96*X_state_cov[:p_n]) 
    plt.plot(X_state_mean[:p_n],'g*-',label="Prediccion de la media") 
    plt.legend()
    plt.title("prediccion de markov zoom en curva")
    plt.show()
    
    
#older = "/work/work_teamEXOPLANET/KOI_LC/"
df_meta=pd.read_csv('../../KOI_Data/kepler_dataset.csv')
df_meta.head()

,Kepler ID,KOI Name,Kepler Name,KOI Number,Kepler Disposition,NExScI Disposition,RA (J2000),RA Error,Dec (J2000),Dec Error,...,Contamination Flag,Insolation Flux,Insolation Flux err1,Insolation Flux err2,Fitted Stellar Density,Fitted Stellar Density err1,Fitted Stellar Density err2,Planetary Fit,Disposition Score,File Name
0,757450,K00889.01,Kepler-75 b,889.01,CANDIDATE,CONFIRMED,19 24 33.024,0.0,+36 34 38.57,0.0,...,0,83.21,16.69,-13.87,11.62061,0.20683,-0.40258,LS+MCMC,0.999,kplr000757450-20160128150956_dvt.fits
1,892772,K01009.01,NaN,1009.01,FALSE POSITIVE,FALSE POSITIVE,19 24 36.811,0.0,+36 40 43.83,0.0,...,0,119.70,48.69,-36.91,0.88510,0.10080,-0.39457,LS+MCMC,0.000,kplr000892772-20160128150956_dvt.fits
2,1025986,K07621.01,NaN,7621.01,FALSE POSITIVE,FALSE POSITIVE,19 24 08.086,0.0,+36 46 15.75,0.0,...,0,2.00,0.92,-0.76,33.55994,16.58298,-31.53180,LS+MCMC,0.000,kplr001025986-20160128150956_dvt.fits
3,1026032,K06252.01,NaN,6252.01,FALSE POSITIVE,FALSE POSITIVE,19 24 10.577,0.0,+36 43 45.38,0.0,...,0,98.60,52.46,-21.18,0.73874,0.00021,-0.00735,LS+MCMC,0.000,kplr001026032-20160128150956_dvt.fits
4,1026957,K00958.01,NaN,958.01,CANDIDATE,CANDIDATE,19 25 01.078,0.0,+36 44 37.00,0.0,...,0,13.18,3.00,-1.98,102.86534,5.47239,-30.25813,LS+MCMC,1.000,kplr001026957-20160128150956_dvt.fits


In [2]:
! ls '/work/work_teamEXOPLANET/KOI_LC/cleaned/'

LC_kepler_processed.npy  LC_kepler_processed_old.npy


In [4]:
confirmados=df_meta.loc[df_meta['NExScI Disposition']=='CONFIRMED']
confirmados

,Kepler ID,KOI Name,Kepler Name,KOI Number,Kepler Disposition,NExScI Disposition,RA (J2000),RA Error,Dec (J2000),Dec Error,...,Contamination Flag,Insolation Flux,Insolation Flux err1,Insolation Flux err2,Fitted Stellar Density,Fitted Stellar Density err1,Fitted Stellar Density err2,Planetary Fit,Disposition Score,File Name
0,757450,K00889.01,Kepler-75 b,889.01,CANDIDATE,CONFIRMED,19 24 33.024,0.0,+36 34 38.57,0.0,...,0,83.21,16.69,-13.87,11.62061,0.20683,-0.40258,LS+MCMC,0.999,kplr000757450-20160128150956_dvt.fits
10,1432789,K00992.01,Kepler-745 b,992.01,CANDIDATE,CONFIRMED,19 25 59.861,0.0,+37 03 33.41,0.0,...,0,177.83,74.92,-54.50,0.48317,0.01261,-0.19180,LS+MCMC,0.848,kplr001432789-20160128150956_dvt.fits
21,1718189,K00993.01,Kepler-262 c,993.01,CANDIDATE,CONFIRMED,19 22 54.881,0.0,+37 15 09.54,0.0,...,0,34.57,26.70,-8.92,2.00195,3.01982,-0.14022,LS+MCMC,1.000,kplr001718189-20160128150956_dvt.fits
22,1718189,K00993.02,Kepler-262 b,993.02,CANDIDATE,CONFIRMED,19 22 54.881,0.0,+37 15 09.54,0.0,...,0,68.66,53.04,-17.72,1.22851,2.18168,-1.19934,LS+MCMC,1.000,kplr001718189-20160128150956_dvt.fits
29,1724719,K04212.01,Kepler-1569 b,4212.01,CANDIDATE,CONFIRMED,19 28 49.130,0.0,+37 13 16.38,0.0,...,0,528.14,248.86,-168.22,0.12599,1.09611,-0.09869,LS+MCMC,0.956,kplr001724719-20160128150956_dvt.fits
31,1725016,K01007.01,Kepler-748 b,1007.01,CANDIDATE,CONFIRMED,19 29 03.098,0.0,+37 14 13.17,0.0,...,0,183.15,171.57,-55.16,1.33381,0.07746,-0.48230,LS+MCMC,1.000,kplr001725016-20160128150956_dvt.fits
34,1865042,K01002.01,Kepler-746 b,1002.01,CANDIDATE,CONFIRMED,19 22 43.740,0.0,+37 21 18.40,0.0,...,0,421.61,157.19,-122.21,5.10370,0.28648,-2.26044,LS+MCMC,0.998,kplr001865042-20160128150956_dvt.fits
38,1871056,K01001.01,Kepler-264 b,1001.01,CANDIDATE,CONFIRMED,19 28 10.709,0.0,+37 22 34.46,0.0,...,0,83.75,38.75,-42.76,0.15947,0.00067,-0.06223,LS+MCMC,0.976,kplr001871056-20160128150956_dvt.fits
39,1871056,K01001.02,Kepler-264 c,1001.02,CANDIDATE,CONFIRMED,19 28 10.709,0.0,+37 22 34.46,0.0,...,0,16.17,7.48,-8.25,0.19495,0.01805,-0.04662,LS+MCMC,0.991,kplr001871056-20160128150956_dvt.fits
41,1873513,K04928.01,Kepler-1624 b,4928.01,CANDIDATE,CONFIRMED,19 30 19.166,0.0,+37 22 35.08,0.0,...,0,20.06,6.53,-1.58,7.28769,0.05143,-0.96909,LS+MCMC,0.998,kplr001873513-20160128150956_dvt.fits


In [5]:
index_co=list(confirmados.index)
len(index_co)

2281

In [6]:
kplr_ids=np.asarray(df_meta['Kepler ID'].values)[[index_co]]
kplr_ids.shape

/users/mbugueno/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


(2281,)

In [7]:
kois_name=np.asarray(df_meta['KOI Name'].values)[[index_co]]
kois_name.shape

/users/mbugueno/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


(2281,)

### recuperando Confirmados 20 estados

In [8]:
our_detrends=np.load('/work/work_teamEXOPLANET/KOI_LC/cleaned/LC_kepler_processed.npy')
print (our_detrends.shape)
our_detrends=our_detrends[index_co]
our_detrends.shape

(8054, 71427)


(2281, 71427)

In [9]:
folder_marg='/work/work_teamEXOPLANET/MTF/'

In [ ]:
import os

i=0
for lc_our_detrend in our_detrends:  
    recuperado=False
    name_kepler=kois_name[i]
    id_kepler=kplr_ids[i]    
    try:
        a=np.load(folder_marg+'confirmados_20/'+str(name_kepler)+'.npy')
        print ("matriz ", name_kepler, 'recuperada previamente')
        i+=1
        continue
    
    except:         
        print ("Calculando matriz para ", name_kepler, "pues tenia kepler id",id_kepler, "con i:",i)
        lc_wind_nan = []
        lengths = []
        sublist = []
        for value in lc_our_detrend:
            if np.isnan(value) and len(sublist) != 0:
                lc_wind_nan.append(np.asarray(sublist))
                lengths.append(len(sublist))
                sublist = []
            elif np.isnan(value) and len(sublist) == 0:
                continue
            else: 
                sublist.append(value) 
        if len(sublist) != 0:      
            lc_wind_nan.append(np.asarray(sublist))
            lengths.append(len(sublist))
        lc_wind_nan = np.asarray(lc_wind_nan)   
        lengths = np.asarray(lengths)   

        lc_wind_nan = np.concatenate(lc_wind_nan).reshape(-1,1)
        lc_wind_nan = lc_wind_nan/np.abs(np.min(lc_wind_nan)) 
        n_sta=20
        markov_model = hmm.GaussianHMM(n_components=n_sta, n_iter=50)
        markov_model.fit(lc_wind_nan , lengths)
        np.save(folder_marg+'confirmados_20/'+str(name_kepler)+'.npy', markov_model.transmat_)
        i+=1


Calculando matriz para  K00889.01 pues tenia kepler id 757450 con i: 0
Calculando matriz para  K00992.01 pues tenia kepler id 1432789 con i: 1
Calculando matriz para  K00993.01 pues tenia kepler id 1718189 con i: 2
Calculando matriz para  K00993.02 pues tenia kepler id 1718189 con i: 3
Calculando matriz para  K04212.01 pues tenia kepler id 1724719 con i: 4
Calculando matriz para  K01007.01 pues tenia kepler id 1725016 con i: 5
Calculando matriz para  K01002.01 pues tenia kepler id 1865042 con i: 6
Calculando matriz para  K01001.01 pues tenia kepler id 1871056 con i: 7
Calculando matriz para  K01001.02 pues tenia kepler id 1871056 con i: 8
Calculando matriz para  K04928.01 pues tenia kepler id 1873513 con i: 9
Calculando matriz para  K02634.01 pues tenia kepler id 1995519 con i: 10
Calculando matriz para  K02534.01 pues tenia kepler id 1996180 con i: 11
Calculando matriz para  K02403.01 pues tenia kepler id 2142522 con i: 12
Calculando matriz para  K00999.01 pues tenia kepler id 2165002

Calculando matriz para  K00624.03 pues tenia kepler id 3541946 con i: 113
Calculando matriz para  K00069.01 pues tenia kepler id 3544595 con i: 114
Calculando matriz para  K02126.01 pues tenia kepler id 3546060 con i: 115
Calculando matriz para  K02194.01 pues tenia kepler id 3548044 con i: 116
Calculando matriz para  K02194.02 pues tenia kepler id 3548044 con i: 117
Calculando matriz para  K01194.01 pues tenia kepler id 3554031 con i: 118
Calculando matriz para  K01194.03 pues tenia kepler id 3554031 con i: 119
Calculando matriz para  K00492.01 pues tenia kepler id 3559935 con i: 120
Calculando matriz para  K03398.01 pues tenia kepler id 3561464 con i: 121
Calculando matriz para  K03418.01 pues tenia kepler id 3629330 con i: 122
Calculando matriz para  K03308.01 pues tenia kepler id 3632089 con i: 123
Calculando matriz para  K03308.02 pues tenia kepler id 3632089 con i: 124
Calculando matriz para  K00975.01 pues tenia kepler id 3632418 con i: 125
Calculando matriz para  K01221.01 pues

Calculando matriz para  K01895.02 pues tenia kepler id 4263293 con i: 224
Calculando matriz para  K01895.03 pues tenia kepler id 4263293 con i: 225
Calculando matriz para  K00551.01 pues tenia kepler id 4270253 con i: 226
Calculando matriz para  K00551.02 pues tenia kepler id 4270253 con i: 227
Calculando matriz para  K03324.01 pues tenia kepler id 4271474 con i: 228
Calculando matriz para  K00813.01 pues tenia kepler id 4275191 con i: 229
Calculando matriz para  K01342.01 pues tenia kepler id 4275721 con i: 230
Calculando matriz para  K01342.02 pues tenia kepler id 4275721 con i: 231
Calculando matriz para  K01615.01 pues tenia kepler id 4278221 con i: 232
Calculando matriz para  K03439.01 pues tenia kepler id 4282191 con i: 233
Calculando matriz para  K01325.01 pues tenia kepler id 4282872 con i: 234
Calculando matriz para  K02652.01 pues tenia kepler id 4346178 con i: 235
Calculando matriz para  K00244.01 pues tenia kepler id 4349452 con i: 236
Calculando matriz para  K00244.02 pues